# Create SPECFEM3D Project
1. Unpickle events and staions
1. Read and map moment tensors to events
1. Prep observed station data for writing to ascii files
1. Write CMTSOLUTION, STATION, STATION_ADJOINT ascii files in appropriate dictories for each event
1. Write adjoint source window parameter files

#### 1. Unpickle events and stations 

In [ ]:
import pickle
from gnam.events.gevents import gevents as gevents
from gnam.events.gstations import gstations as gstations
from gnam.events.munge.knmi import correct_station_depths as csd_f

data_root_dir = '../../../data_notebooks'

#Unpickle events
print('Unpickling Events')
f = open('../../../data_notebooks/pickled/events.pickle', 'rb')
dill_events = pickle.load(f)
f.close()
    

#Unpickle stations
print('Unpickling Station Traces')
f = open('../../../data_notebooks/pickled/straces.pickle', 'rb')
dill_straces = pickle.load(f)
f.close()

# This is kind of hokey, but it works for now.
# Some of the stations depths do not follow the 
# 50, 100, 150, 200 meter depths -- possibly because
# the boreholes are slanted. To correct for this,
# a hard coded "patch/update" is applied. See the
# code for details and update values.
dill_straces.correct_stations(csd_f)

print('Done')

#### 2. Read and map moment tensors to events

In [ ]:
from gnam.events.mtensors import mtensors
import pandas as pd
import numpy as np

#Get model bounding box needed for adjusting strike
gf_bbox = dill_events.getBBox()

#Read moment tensors
#gf_mts = mtensors('../../../data_notebooks/data/event_moments.csv')
gf_mts = mtensors('../../../data_notebooks/data/event_moments.csv',gf_bbox)
    
# get event catalog of the events withing the bounding box
e_cat = dill_events.getIncCatalog()

# This is a bit hokey, but it works. Here we update the
# event time from the moment tensor CSV file with thouse
# from the event catalog
gf_mts.update_utcdatetime(e_cat)
for imt in range(len(gf_mts)):
    print("Moment-Tensor %d:\n" %(imt),gf_mts[imt])

# Create a dictionary that maps moment tensors to events
e2mt_dict = gf_mts.map_events_2_tensors(e_cat)
e2mt_keys = e2mt_dict.keys()

# Print a comparison of events to moment tensors
for key in e2mt_keys:
    print('UTC: event[%d][Date] = %s' %(key,e_cat[key].origins[0].time))
    print('UTC:    MT[%d][Date] = %s' %(key,e2mt_dict[key]['Date']))
    print('Mag: event[%d][Date] = %s' %(key,e_cat[key].magnitudes[0].mag))
    print('Mag:    MT[%d][Date] = %s' %(key,e2mt_dict[key]['ML']))
    print()

#### 3. Prep observed station data for writing to ascii files

In [ ]:
from obspy import Stream

#create streams and map eventid to stream
e_stream_dict = {}
ce_stream_dict = {}
for ekey in e2mt_keys:
    '''
    e_st  = Stream()
    e_st += dill_straces.getStreamZ(ekey,3) 
    e_st += dill_straces.getStreamZ(ekey,4) 
    e_st.resample(1000)
    e_stream_dict[ekey] = e_st
    '''
    
    #get boreholes 3 and 4 streams for the event
    e1_st3 = dill_straces.getStream1(ekey,3)
    e1_st4 = dill_straces.getStream1(ekey,4)
    e2_st3 = dill_straces.getStream2(ekey,3)
    e2_st4 = dill_straces.getStream2(ekey,4)
    ez_st3 = dill_straces.getStreamZ(ekey,3)
    ez_st4 = dill_straces.getStreamZ(ekey,4)

    #combine component streams
    e_st3 = e1_st3 + e2_st3 + ez_st3
    e_st4 = e1_st4 + e2_st4 + ez_st4

    #reorder streams so that each station component is contiguous
    te_st3 = Stream()
    te_st4 = Stream()

    ns = len(ez_st3)
    for i in range(ns):
        te_st3 += e_st3[i+2*ns]
        te_st3 += e_st3[i+ns]
        te_st3 += e_st3[i]

        te_st4 += e_st4[i+2*ns]
        te_st4 += e_st4[i+ns]
        te_st4 += e_st4[i]
    
    e_st3 = te_st3
    e_st4 = te_st4
    
    #rotate the streams to ZNE components (obspy and the FDSN info is used)
    inv3 = dill_straces.get_inventory(3)
    inv4 = dill_straces.get_inventory(4)
    e_st3.rotate(method="->ZNE", inventory=inv3)
    e_st4.rotate(method="->ZNE", inventory=inv4)

    #resample and map event to stream
    e_st = e_st3 + e_st4
    ce_st = e_st.copy() #keep a pre-resampled set for P-arrival picking (less memory)
    #e_st.resample(1000) #better to resample while iterating. 
    e_stream_dict[ekey] = e_st
    ce_stream_dict[ekey] = ce_st

#### 4.  Setup and test DATA directories and specfem3d file names

In [ ]:
experiments = []
for ekey in e2mt_keys:
    bkeys = dill_straces.getBoreholeKeys()
    bh_stat_list = []
    for bkey in bkeys:
        bh_stat_list.append(dill_straces.getIncludedStations(ekey,bkey))
    bh_stat_tup = tuple(bh_stat_list)   
    exp_set = {'EKEY':ekey,'EVENT':e_cat[ekey],'TENSOR':e2mt_dict[ekey],'STATIONS':bh_stat_tup}
    experiments.append(exp_set)

dir_set_list = []
min_eid = experiments[0]['EKEY']
for eset in experiments:
    eid = eset['EKEY']
    if eid < min_eid:
        min_eid = eid
        
dir_event_list = []
for eset in experiments:
    rdir = eset['EKEY'] - min_eid + 1
    rdir = 'run%s' %(str(rdir).zfill(4))
    dname = data_root_dir + '/SPECFEM3D/project/%s' %(rdir)
    dir_event_list.append((eset,dname))
    print('Experiment-%d:\n'%(eset['EKEY']), dname)
    print()
dir_event_list = dir_event_list[::-1]

#### 5.  Create directories and write CMTSOLUTION, STATION, STATION_ADJOINT, and trace ascii files in separate dictories for each event

In [ ]:
en = 3 #last event (really it's: en-1)

In [ ]:
from gnam.specutils.write import write_cmtsolution as spec_wcmt
from gnam.specutils.write import write_stations as spec_ws
from gnam.specutils.write import write_stream_2_spec_ascii as spec_wst2a
import os

nproc = 16

# define the access rights
access_rights = 0o755

#loop over dir-set pairs
for pair in dir_event_list[0:en]:
    eset  = pair[0]
    dname = pair[1]
    new_dir = dname + '/DATA'
    print('Experiment-%d:\n'%(eset['EKEY']), dname)
    try:
        os.makedirs(new_dir, access_rights)
    except OSError:
        print ("Creation of the directory %s failed" % new_dir)
        print()
    else:
        bkeys = dill_straces.getBoreholeKeys()
        lxy = dill_events.getLocalEventCoord(eset['EKEY'])
        spec_wcmt(new_dir + '/CMTSOLUTION',eset['TENSOR'],lxy[0],lxy[1])
        spec_ws(new_dir + '/STATIONS',dill_straces,eset['EKEY'],bkeys)
        spec_ws(new_dir + '/STATIONS_ADJOINT',dill_straces,eset['EKEY'],bkeys)
        
        #add links to bin and utils
        src = '/quanta1/home/tcullison/DevGPU_specfem3d/bin'
        dst = dname + '/bin'
        os.symlink(src, dst)
        src = '/quanta1/home/tcullison/DevGPU_specfem3d/utils'
        dst = dname + '/utils'
        os.symlink(src, dst)
        
    #add OUTPUT_FILES/DATABASES_MPI dirs
    new_dir  = dname + '/OUTPUT_FILES/DATABASES_MPI'
    try:
        os.makedirs(new_dir, access_rights)
    except OSError:
        print ("Creation of the directory %s failed" % new_dir)
        print()
    else:
        if eset['EKEY'] != min_eid:
            #for k in ['Database','vp.bin','vs.bin','rho.bin']:
            for k in ['Database','external_mesh.bin']:
                for i in range(nproc):
                    iproc = str(i).zfill(6)
                    mesh_fname = 'proc%s_%s' %(iproc,k)
                    src = '../../../run0001/OUTPUT_FILES/DATABASES_MPI' + mesh_fname
                    dst = new_dir + '/' + mesh_fname
                    os.symlink(src, dst)
            mesh_fname = 'surface_from_mesher.h'
            src = '../../../run0001/OUTPUT_FILES/' + mesh_fname
            dst = dname + '/OUTPUT_FILES/' + mesh_fname
            os.symlink(src, dst)
            mesh_fname = 'values_from_mesher.h'
            src = '../../run0001/OUTPUT_FILES/' + mesh_fname
            dst = dname + '/OUTPUT_FILES/' + mesh_fname
            os.symlink(src, dst)
    
    #add OBS dir
    new_dir  = dname + '/OBS'
    try:
        os.makedirs(new_dir, access_rights)
    except OSError:
        print ("Creation of the directory %s failed" % new_dir)
        print()
    else:
        eid = eset['EKEY']
        e_st = e_stream_dict[eid]
        '''
        for tr in e_st:
            spec_pair_list = [] #time,amplitude pairs

            #Example of filename: 'NL.G094.FXX.semd'
            tr_filename = 'NL.' + tr.stats.station + '.FXZ.semd'
            print('filename:',tr_filename)

            for it in range(tr.count()):
                spec_pair_list.append('%E   %E\n' %(tr.times()[it],tr.data[it]))

            fqpname = new_dir + '/' + tr_filename
            f = open(fqpname, 'w')
            f.writelines(spec_pair_list)
            f.close()
            #Example of filename: 'NL.G094.FXX.semd'
            tr_spec_chan = '.BOO.'
            comp_char = tr.stats.channel[2] 
            if comp_char == 'Z':
                tr_spec_chan = '.FXZ.'
            elif comp_char == 'E':
                tr_spec_chan = '.FXX.'
            elif comp_char == 'N':
                tr_spec_chan = '.FXY.'
            else:
                print('Uh-oh! Spaghetti Os!')

            tr_filename = 'NL.' + tr.stats.station + tr_spec_chan + 'semd'

            for it in range(tr.count()):
                spec_pair_list.append('%E   %E\n' %(tr.times()[it],tr.data[it]))

            fqpname = new_dir + '/' + tr_filename
            f = open(fqpname, 'w')
            f.writelines(spec_pair_list)
            f.close()
        '''
        #write specfem3d ascii files 
        spec_wst2a(e_st,new_dir,is_zne=True) 
        
        
    #add FILT_OBS dir
    new_dir  = dname + '/FILT_OBS'
    try:
        os.makedirs(new_dir, access_rights)
    except OSError:
        print ("Creation of the directory %s failed" % new_dir)
        print()
        
        
    #add SYN dir
    new_dir  = dname + '/SYN'
    try:
        os.makedirs(new_dir, access_rights)
    except OSError:
        print ("Creation of the directory %s failed" % new_dir)
        print()
        
        
    #add FILT_SYN dir
    new_dir  = dname + '/FILT_SYN'
    try:
        os.makedirs(new_dir, access_rights)
    except OSError:
        print ("Creation of the directory %s failed" % new_dir)
        print()
        
        
    #add SEM dir
    new_dir  = dname + '/SEM'
    try:
        os.makedirs(new_dir, access_rights)
    except OSError:
        print ("Creation of the directory %s failed" % new_dir)
        print()
        
        
    #if run0001 dir (this is were the model setup and update happens)
    if eset['EKEY'] == min_eid:
        
        #add INPUT_KERNELS dir
        new_dir  = dname + '/INPUT_KERNELS'
        try:
            os.makedirs(new_dir, access_rights)
        except OSError:
            print ("Creation of the directory %s failed" % new_dir)
            print()
        
        #add INPUT_MODEL dir
        new_dir  = dname + '/INPUT_MODEL'
        try:
            os.makedirs(new_dir, access_rights)
        except OSError:
            print ("Creation of the directory %s failed" % new_dir)
            print()
        
        #add OUTPUT_MODEL dir
        new_dir  = dname + '/OUTPUT_MODEL'
        try:
            os.makedirs(new_dir, access_rights)
        except OSError:
            print ("Creation of the directory %s failed" % new_dir)
            print()
        
        #add OUTPUT_SUM dir
        new_dir  = dname + '/OUTPUT_SUM'
        try:
            os.makedirs(new_dir, access_rights)
        except OSError:
            print ("Creation of the directory %s failed" % new_dir)
            print()
        
        #add SMOOTH dir
        new_dir  = dname + '/SMOOTH'
        try:
            os.makedirs(new_dir, access_rights)
        except OSError:
            print ("Creation of the directory %s failed" % new_dir)
            print()
        else:
            for k in ['alpha','beta','rho','hess']:
                for i in range(nproc):
                    iproc = str(i).zfill(6)
                    kernel_fname = 'proc%s_%s_kernel' %(iproc,k)
                    src = './' + kernel_fname + '_smooth.bin'
                    dst = new_dir + '/' + kernel_fname + '.bin'
                    os.symlink(src, dst)
        
        #add INPUT_GRADIENT dir
        new_dir  = dname + '/INPUT_GRADIENT'
        try:
            os.makedirs(new_dir, access_rights)
        except OSError:
            print ("Creation of the directory %s failed" % new_dir)
            print()
        else:
            for k in ['alpha','beta','rho','hess']:
                for i in range(nproc):
                    iproc = str(i).zfill(6)
                    kernel_fname = 'proc%s_%s_kernel_smooth' %(iproc,k)
                    src = '../SMOOTH/' + kernel_fname + '.bin'
                    dst = new_dir + '/' + kernel_fname + '.bin'
                    os.symlink(src, dst)
            
        #add topo dir
        new_dir  = dname + '/topo'
        try:
            os.makedirs(new_dir, access_rights)
        except OSError:
            print ("Creation of the directory %s failed" % new_dir)
            print()
        else:
            for i in range(nproc):
                iproc = str(i).zfill(6)
                ext_mesh_fname = 'proc%s_external_mesh.bin' %(iproc)
                src = '../OUTPUT_FILES/DATABASES_MPI/' + ext_mesh_fname
                dst = new_dir + '/' + ext_mesh_fname
                os.symlink(src, dst)
                
        #add COMBINE dir (empty)
        new_dir  = dname + '/COMBINE'
        try:
            os.makedirs(new_dir, access_rights)
        except OSError:
            print ("Creation of the directory %s failed" % new_dir)
            print()


#### 9. Write adjoint source window parameter files 

In [ ]:
import obspy
from obspy import Stream
from obspy import Trace
from obspy import UTCDateTime


#loop over dir-set pairs
for pair in dir_event_list[0:en]:
    eset  = pair[0]
    dname = pair[1]
    eid = eset['EKEY']
    trig_dict = event_trigger_dict[eid]
    
    t1 = 2
    for key in trig_dict:

        tr_filename = 'NL.' + key + '.window'

        p_trig = trig_dict[key][0]
        p0     = trig_dict[key][1]
        s0     = trig_dict[key][2]
        s_trig = trig_dict[key][3]
        str_t0 = 'T0 = %.3f\n' %(p_trig)
        str_t1 = 'T1 = %.3f\n' %(s_trig)
        str_p0 = 'P0 = %.3f\n' %(p0)
        str_s0 = 'S0 = %.3f\n' %(s0)
        window_list = [str_t0,str_t1,str_p0,str_s0] 

        #print('Filename:', tr_filename)
        #print('Contents:\n', window_list)

        fqpname = dname + '/OBS/' + tr_filename
        #print('fqn:',fqpname)
        f = open(fqpname, 'w')
        f.writelines(window_list)
        f.close()

## Finished